In [3]:
# USE sklearn 1.0.2 and sklearn-deap 0.3.0 (git clone and python setup.py install) For task EvolutionaryAlgorithmSearchCV

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.svm import SVC,SVR
from sklearn import datasets
import scipy.stats as stats


/home/test/anaconda3/envs/astrophysics-sklearn-deap/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
d = datasets.load_digits()
X = d.data
y = d.target

In [5]:
datasets.load_digits()

{'data': array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ..., 10.,  0.,  0.],
        [ 0.,  0.,  0., ..., 16.,  9.,  0.],
        ...,
        [ 0.,  0.,  1., ...,  6.,  0.,  0.],
        [ 0.,  0.,  2., ..., 12.,  0.,  0.],
        [ 0.,  0., 10., ..., 12.,  1.,  0.]]),
 'target': array([0, 1, 2, ..., 8, 9, 8]),
 'frame': None,
 'feature_names': ['pixel_0_0',
  'pixel_0_1',
  'pixel_0_2',
  'pixel_0_3',
  'pixel_0_4',
  'pixel_0_5',
  'pixel_0_6',
  'pixel_0_7',
  'pixel_1_0',
  'pixel_1_1',
  'pixel_1_2',
  'pixel_1_3',
  'pixel_1_4',
  'pixel_1_5',
  'pixel_1_6',
  'pixel_1_7',
  'pixel_2_0',
  'pixel_2_1',
  'pixel_2_2',
  'pixel_2_3',
  'pixel_2_4',
  'pixel_2_5',
  'pixel_2_6',
  'pixel_2_7',
  'pixel_3_0',
  'pixel_3_1',
  'pixel_3_2',
  'pixel_3_3',
  'pixel_3_4',
  'pixel_3_5',
  'pixel_3_6',
  'pixel_3_7',
  'pixel_4_0',
  'pixel_4_1',
  'pixel_4_2',
  'pixel_4_3',
  'pixel_4_4',
  'pixel_4_5',
  'pixel_4_6',
  'pixel_4_7',
  'pixel_5_0',
  'pixel_5_1',
 

In [5]:
#Random Forest
from evolutionary_search import EvolutionaryAlgorithmSearchCV
# Define the hyperparameter configuration space
rf_params = {
    'n_estimators': np.logspace(1,1.8,num = 10 ,base=20,dtype='int'),
    'max_depth': np.logspace(1,2,num = 10 ,base=10,dtype='int'),
    "max_features":np.logspace(0.2,1,num = 5 ,base=8,dtype='int'),
    "min_samples_split":np.logspace(0.4, 1, num=5, base=10, dtype='int'), #[2, 3, 5, 7, 10],
    "min_samples_leaf":np.logspace(0.1,1,num = 5 ,base=11,dtype='int'),
    "criterion":['gini','entropy']
}
rf_params = {
    'n_estimators': range(10,100),
    "max_features":range(1,64),
    'max_depth': range(5,50),
    "min_samples_split":range(2,11),
    "min_samples_leaf":range(1,11),
    #Categorical(name='criterion', categories=['gini','entropy'])#
    "criterion":['gini','entropy']
}
clf = RandomForestClassifier(random_state=0)
# Set the hyperparameters of GA 
ga1 = EvolutionaryAlgorithmSearchCV(estimator=clf,
                                   params=rf_params,
                                   scoring="accuracy",
                                   cv=3,
                                   verbose=1,
                                   population_size=10,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=5,
                                   n_jobs=1)
ga1.fit(X, y)
print(ga1.best_params_)
print("Accuracy:"+ str(ga1.best_score_))

/home/test/anaconda3/envs/astrophysics-sklearn-deap/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/test/anaconda3/envs/astrophysics-sklearn-deap/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Types [1, 1, 1, 1, 1, 1] and maxint [89, 62, 44, 8, 9, 1] detected
--- Evolve in 45927000 possible combinations ---
gen	nevals	avg     	min     	max     	std      
0  	10    	0.894602	0.873122	0.926544	0.0158167
1  	7     	0.913244	0.893712	0.926544	0.0126884
2  	3     	0.921425	0.896494	0.92877 	0.00958638
3  	8     	0.926878	0.926544	0.92877 	0.000712646
4  	6     	0.926711	0.913189	0.931553	0.00470911 
5  	6     	0.928325	0.926544	0.92877 	0.000738258
Best individual is: {'n_estimators': 70, 'max_features': 3, 'max_depth': 11, 'min_samples_split': 9, 'min_samples_leaf': 1, 'criterion': 'entropy'}
with fitness: 0.9315525876460768
{'n_estimators': 70, 'max_features': 3, 'max_depth': 11, 'min_samples_split': 9, 'min_samples_leaf': 1, 'criterion': 'entropy'}
Accuracy:0.9315525876460768


In [6]:
#SVM
from evolutionary_search import EvolutionaryAlgorithmSearchCV
rf_params = {
    'C': np.random.uniform(0,50,1000),
    "kernel":['linear','poly','rbf','sigmoid']
}
clf = SVC(gamma='scale')
ga1 = EvolutionaryAlgorithmSearchCV(estimator=clf,
                                   params=rf_params,
                                   scoring="accuracy",
                                   cv=3,
                                   verbose=1,
                                   population_size=10,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=5,
                                   n_jobs=1)
ga1.fit(X, y)
print(ga1.best_params_)
print("Accuracy:"+ str(ga1.best_score_))


/home/test/anaconda3/envs/astrophysics-sklearn-deap/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/test/anaconda3/envs/astrophysics-sklearn-deap/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Types [2, 1] and maxint [999, 3] detected
--- Evolve in 4000 possible combinations ---
gen	nevals	avg    	min     	max     	std      
0  	10    	0.94246	0.763495	0.973845	0.0610608
1  	8     	0.97084	0.943795	0.973845	0.00901503
2  	4     	0.972677	0.962159	0.973845	0.00350584
3  	7     	0.973845	0.973845	0.973845	0         
4  	1     	0.973901	0.973845	0.974402	0.000166945
5  	6     	0.974068	0.973845	0.974402	0.00027262 
Best individual is: {'C': 6.730646889908404, 'kernel': 'rbf'}
with fitness: 0.9744017807456873
{'C': 6.730646889908404, 'kernel': 'rbf'}
Accuracy:0.9744017807456873


In [10]:
#KNN
from evolutionary_search import EvolutionaryAlgorithmSearchCV
rf_params = {
    'n_neighbors': range(1,20),
}
clf = KNeighborsClassifier()
ga1 = EvolutionaryAlgorithmSearchCV(estimator=clf,
                                   params=rf_params,
                                   scoring="accuracy",
                                   cv=3,
                                   verbose=1,
                                   population_size=10,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=5,
                                   n_jobs=1)
ga1.fit(X, y)
print(ga1.best_params_)
print("Accuracy:"+ str(ga1.best_score_))

/home/test/anaconda3/envs/astrophysics-sklearn-deap/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/test/anaconda3/envs/astrophysics-sklearn-deap/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Types [1] and maxint [18] detected
--- Evolve in 19 possible combinations ---
gen	nevals	avg     	min     	max     	std       
0  	10    	0.954869	0.945465	0.963829	0.00638599
1  	3     	0.959265	0.947691	0.963829	0.00502563
2  	8     	0.961603	0.956594	0.963829	0.00252572
3  	5     	0.962827	0.962716	0.963829	0.00033389
4  	6     	0.962938	0.962716	0.963829	0.000445186
5  	8     	0.961658	0.954368	0.963829	0.00305255 
Best individual is: {'n_neighbors': 4}
with fitness: 0.9638286032276016
{'n_neighbors': 4}
Accuracy:0.9638286032276016


In [11]:
#ANN
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping
def ANN(optimizer = 'sgd',neurons=32,batch_size=32,epochs=20,activation='relu',patience=3,loss='categorical_crossentropy'):
    model = Sequential()
    model.add(Dense(neurons, input_shape=(X.shape[1],), activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(10,activation='softmax'))  # 10 is the number of classes in the dataset, you can change it based on your dataset
    model.compile(optimizer = optimizer, loss=loss)
    early_stopping = EarlyStopping(monitor="loss", patience = patience)# early stop patience
    history = model.fit(X, pd.get_dummies(y).values,
              batch_size=batch_size,
              epochs=epochs,
              callbacks = [early_stopping],
              verbose=0) #verbose set to 1 will show the training process
    return model


from evolutionary_search import EvolutionaryAlgorithmSearchCV
# Define the hyperparameter configuration space
rf_params = {
    'optimizer': ['adam','rmsprop','sgd'],
    'activation': ['relu','tanh'],
    'batch_size': [16,32,64],
    'neurons':range(10,100),
    'epochs':[20,50],
    #'epochs':[20,50,100,200],
    'patience':range(3,20)
}
clf = KerasClassifier(build_fn=ANN, verbose=0)
# Set the hyperparameters of GA    
ga1 = EvolutionaryAlgorithmSearchCV(estimator=clf,
                                   params=rf_params,
                                   scoring="accuracy",
                                   cv=3,
                                   verbose=1,
                                   population_size=10,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=5,
                                   n_jobs=1)
ga1.fit(X, y)
print(ga1.best_params_)
print("Accuracy:"+ str(ga1.best_score_))

/home/test/anaconda3/envs/astrophysics-sklearn-deap/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/test/anaconda3/envs/astrophysics-sklearn-deap/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Types [1, 1, 1, 1, 1, 1] and maxint [2, 1, 2, 89, 1, 16] detected
--- Evolve in 55080 possible combinations ---
gen	nevals	avg     	min     	max	std      
0  	10    	0.975849	0.933779	1  	0.0230376
1  	6     	0.993656	0.967724	1  	0.00955532
2  	7     	0.999388	0.997774	1  	0.000946021
3  	4     	1       	1       	1  	0          
4  	6     	1       	1       	1  	0          
5  	7     	1       	1       	1  	0          
Best individual is: {'optimizer': 'adam', 'activation': 'relu', 'batch_size': 32, 'neurons': 82, 'epochs': 50, 'patience': 7}
with fitness: 1.0
{'optimizer': 'adam', 'activation': 'relu', 'batch_size': 32, 'neurons': 82, 'epochs': 50, 'patience': 7}
Accuracy:1.0


In [1]:
### Using TPOT


In [6]:
#Random Forest
from tpot import TPOTClassifier
# Define the hyperparameter configuration space
parameters = {
        'n_estimators': range(20,200),
    "max_features":range(1,64),
    'max_depth': range(10,100),
    "min_samples_split":range(2,11),
    "min_samples_leaf":range(1,11),
    "criterion":['gini','entropy']
             }
# Set the hyperparameters of GA                 
ga2 = TPOTClassifier(generations= 3, population_size= 10, offspring_size= 5,
                                 verbosity= 3, early_stop= 5,
                                 config_dict=
                                 {'sklearn.ensemble.RandomForestClassifier': parameters}, 
                                 cv = 3, scoring = 'accuracy')
ga2.fit(X, y)

1 operators have been imported by TPOT.



Generation 1 - Current Pareto front scores:

-1	0.9454646633277685	RandomForestClassifier(CombineDFs(input_matrix, input_matrix), RandomForestClassifier__criterion=entropy, RandomForestClassifier__max_depth=97, RandomForestClassifier__max_features=5, RandomForestClassifier__min_samples_leaf=1, RandomForestClassifier__min_samples_split=3, RandomForestClassifier__n_estimators=179)

Generation 2 - Current Pareto front scores:

-1	0.9454646633277685	RandomForestClassifier(CombineDFs(input_matrix, input_matrix), RandomForestClassifier__criterion=entropy, RandomForestClassifier__max_depth=97, RandomForestClassifier__max_features=5, RandomForestClassifier__min_samples_leaf=1, RandomForestClassifier__min_samples_split=3, RandomForestClassifier__n_estimators=179)

Generation 3 - Current Pareto front scores:

-1	0.9454646633277685	RandomForestClassifier(CombineDFs(input_matrix, input_matrix), RandomForestClassifier__criterion=entropy, RandomForestClassifier__max_depth=97, RandomForestClassifier

TPOTClassifier(config_dict={'sklearn.ensemble.RandomForestClassifier': {'criterion': ['gini',
                                                                                      'entropy'],
                                                                        'max_depth': range(10, 100),
                                                                        'max_features': range(1, 64),
                                                                        'min_samples_leaf': range(1, 11),
                                                                        'min_samples_split': range(2, 11),
                                                                        'n_estimators': range(20, 200)}},
               cv=3, early_stop=5, generations=3, offspring_size=5,
               population_size=10, scoring='accuracy', verbosity=3)

In [7]:
#SVM
from tpot import TPOTClassifier

parameters = {
    'C': np.random.uniform(0,50,1000),
    "kernel":['linear','poly','rbf','sigmoid']
             }
               
ga2 = TPOTClassifier(generations= 3, population_size= 10, offspring_size= 5,
                                 verbosity= 3, early_stop= 5,
                                 config_dict=
                                 {'sklearn.svm.SVC': parameters}, 
                                 cv = 3, scoring = 'accuracy')
ga2.fit(X, y)


1 operators have been imported by TPOT.



Generation 1 - Current Pareto front scores:

-1	0.9738452977184195	SVC(CombineDFs(input_matrix, input_matrix), SVC__C=49.8486617673746, SVC__kernel=rbf)

Generation 2 - Current Pareto front scores:

-1	0.9738452977184195	SVC(CombineDFs(input_matrix, input_matrix), SVC__C=49.8486617673746, SVC__kernel=rbf)
Pipeline encountered that has previously been evaluated during the optimization process. Using the score from the previous evaluation.

Generation 3 - Current Pareto front scores:

-1	0.9738452977184195	SVC(CombineDFs(input_matrix, input_matrix), SVC__C=49.8486617673746, SVC__kernel=rbf)


TPOTClassifier(config_dict={'sklearn.svm.SVC': {'C': array([5.10688833e+00, 4.99068702e+01, 3.42268065e+01, 2.93952246e+00,
       4.27541697e-01, 4.81218196e+01, 1.66148413e+01, 3.00756992e-01,
       4.94774433e+01, 3.87947769e+01, 3.09684589e+00, 1.54952320e+01,
       2.07866181e+01, 1.28038842e+01, 1.92360590e+01, 4.92571547e+01,
       2.80974955e+01, 1.64139467e+00, 3.29079470e+01, 6.46908...
       2.98006653e+01, 2.03259444e+01, 1.61195448e+01, 5.75471731e+00,
       1.96097420e+01, 1.15769103e+01, 3.38836913e+01, 3.31056785e+01,
       5.76068848e+00, 1.99586594e+01, 3.42079696e+01, 3.83804909e+01,
       2.77333514e+01, 1.72032263e+01, 4.60346408e+01, 4.02337723e+01]),
                                                'kernel': ['linear', 'poly',
                                                           'rbf', 'sigmoid']}},
               cv=3, early_stop=5, generations=3, offspring_size=5,
               population_size=10, scoring='accuracy', verbosity=3)

In [8]:
#KNN
from tpot import TPOTClassifier

parameters = {
    'n_neighbors': range(1,20),
             }
               
ga2 = TPOTClassifier(generations= 3, population_size= 10, offspring_size= 5,
                                 verbosity= 3, early_stop= 5,
                                 config_dict=
                                 {'sklearn.neighbors.KNeighborsClassifier': parameters}, 
                                 cv = 3, scoring = 'accuracy')
ga2.fit(X, y)

1 operators have been imported by TPOT.



Generation 1 - Current Pareto front scores:

-1	0.9638286032276016	KNeighborsClassifier(input_matrix, KNeighborsClassifier__n_neighbors=1)

-2	0.9654980523094046	KNeighborsClassifier(KNeighborsClassifier(input_matrix, KNeighborsClassifier__n_neighbors=17), KNeighborsClassifier__n_neighbors=4)

Generation 2 - Current Pareto front scores:

-1	0.9682804674457429	KNeighborsClassifier(input_matrix, KNeighborsClassifier__n_neighbors=3)

Generation 3 - Current Pareto front scores:

-1	0.9682804674457429	KNeighborsClassifier(input_matrix, KNeighborsClassifier__n_neighbors=3)


TPOTClassifier(config_dict={'sklearn.neighbors.KNeighborsClassifier': {'n_neighbors': range(1, 20)}},
               cv=3, early_stop=5, generations=3, offspring_size=5,
               population_size=10, scoring='accuracy', verbosity=3)